In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, count, last, lag, row_number, substring_index, input_file_name, regexp_extract, expr, concat_ws, map_from_arrays
from pyspark.sql.window import Window

In [2]:
sp = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/24 13:58:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
src_df = sp.read.csv('csv', header=True)\
   .withColumn(
      "tweet_date", 
      regexp_extract(input_file_name(), "/(\d{4}-\d{2}-\d{2})", 1)
   )\
   .withColumn('hashtags', expr(r"regexp_extract_all(text, '(#\\w+)', 0)") )

src_df.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `text` cannot be resolved. Did you mean one of the following? [`_c0`, `_c1`, `_c2`, `_c3`, `_c4`].; line 1 pos 19;
'Project [_c0#988, _c1#989, _c2#990, _c3#991, _c4#992, _c5#993, _c6#994, _c7#995, _c8#996, _c9#997, _c10#998, _c11#999, _c12#1000, _c13#1001, _c14#1002, _c15#1003, _c16#1004, _c17#1005, _c18#1006, _c19#1007, _c20#1008, _c21#1009, tweet_date#1032, 'regexp_extract_all('text, (#\w+), 0) AS hashtags#1057]
+- Project [_c0#988, _c1#989, _c2#990, _c3#991, _c4#992, _c5#993, _c6#994, _c7#995, _c8#996, _c9#997, _c10#998, _c11#999, _c12#1000, _c13#1001, _c14#1002, _c15#1003, _c16#1004, _c17#1005, _c18#1006, _c19#1007, _c20#1008, _c21#1009, regexp_extract(input_file_name(), /(\d{4}-\d{2}-\d{2}), 1) AS tweet_date#1032]
   +- Relation [_c0#988,_c1#989,_c2#990,_c3#991,_c4#992,_c5#993,_c6#994,_c7#995,_c8#996,_c9#997,_c10#998,_c11#999,_c12#1000,_c13#1001,_c14#1002,_c15#1003,_c16#1004,_c17#1005,_

In [4]:
# q1

derived_df1 = src_df.select(
   "tweet_date",
   "user_id",
   "hashtags",
   "source",
   "screen_name",
   "created_at",
).withColumn(
   "screen_name_last", 
   last("screen_name")\
      .over(
         Window.partitionBy("user_id", "tweet_date").orderBy(col("created_at").desc())
      )
)

derived_df1.show()


+----------+-------+--------+------+-------------------+--------------------+-------------------+
|tweet_date|user_id|hashtags|source|        screen_name|          created_at|   screen_name_last|
+----------+-------+--------+------+-------------------+--------------------+-------------------+
|2020-04-12|   NULL|    NULL|  NULL|               NULL|                  😢|               NULL|
|2020-04-12|   NULL|    NULL|  NULL|               NULL|       स्वस्थ रहे 🙏|               NULL|
|2020-04-12|   NULL|    NULL|  NULL|               NULL|dedikten sonra bi...|               NULL|
|2020-04-12|   NULL|    NULL|  NULL|               NULL|       ............ |               NULL|
|2020-04-12|   NULL|    NULL|  NULL|               NULL|                   .|               NULL|
|2020-04-12|   NULL|    NULL|  NULL|               NULL|,Twitter for iPad...|               NULL|
|2020-04-12|   NULL|    NULL|  NULL|               NULL|                   !|               NULL|
|2020-04-12|   NULL|  

In [60]:
derived_df11 = derived_df1\
.groupBy(
   "tweet_date",
   "user_id",
   "screen_name_last",
).agg(
   count("*").alias("num_tweets"),
   collect_list("source").alias("source_list"),
   collect_list("hashtags").alias("hashtags_list")
)

derived_df11 = derived_df11\
   .withColumn(
      "source_map", 
      map_from_arrays(col("source_list"), count(col("source_list")))
)

derived_df11.show()

+----------+--------------------+--------------------+----------+--------------------+-------------+
|tweet_date|             user_id|    screen_name_last|num_tweets|         source_list|hashtags_list|
+----------+--------------------+--------------------+----------+--------------------+-------------+
|2020-03-29|   &amp; Committe...|                NULL|         1|                  []|           []|
|2020-03-29|           +   3'042|                NULL|         1|                  []|           []|
|2020-03-29|  #chiaraferragni...| 1244352573676703744|         1|          [EsterSgb]|         [[]]|
|2020-03-29|  17 se han recup...| Twitter for Android|         1|                  []|           []|
|2020-03-29|         29 fighters|                NULL|         1|                  []|           []|
|2020-03-29|           AC Milan.|                NULL|         1|                  []|           []|
|2020-03-29|  Call 111-111-88...|                NULL|         1|                  []|     